<a href="https://colab.research.google.com/github/hzhang0110/hzhang_ML_Projects/blob/main/makemore_mlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [295]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt #for charts


In [296]:
#read all words
words = open('/content/drive/MyDrive/machine_learning_files/names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [297]:
len(words)

32033

In [298]:
#build the vocabulary of characters and mapping to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(stoi)
print(itos)


{'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, '.': 0}
{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [299]:
#build the dataset

block_size = 3
X,Y = [], []
for w in words:

  #print(w)
  context = [0] * block_size
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    #print(''.join(itos[i] for i in context), '--->', itos[ix])
    context = context[1:] + [ix] #crop and append

X = torch.tensor(X)
Y = torch.tensor(Y)


In [300]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([228146, 3]), torch.int64, torch.Size([228146]), torch.int64)

In [301]:
X, Y

(tensor([[ 0,  0,  0],
         [ 0,  0,  5],
         [ 0,  5, 13],
         ...,
         [26, 26, 25],
         [26, 25, 26],
         [25, 26, 24]]),
 tensor([ 5, 13, 13,  ..., 26, 24,  0]))

In [302]:
C = torch.randn((27,2))
C

tensor([[ 0.9722,  0.1589],
        [ 0.8976, -0.2782],
        [-0.1007, -0.4838],
        [-0.2527, -1.1952],
        [ 0.8320,  0.7421],
        [-0.6390,  1.1987],
        [-0.3974,  0.3412],
        [ 1.9598,  0.8498],
        [ 0.9134, -0.7378],
        [ 0.6476, -0.5303],
        [ 0.3735,  0.5088],
        [ 0.9197,  0.7369],
        [-0.2632, -0.7998],
        [-0.8901,  1.7070],
        [-2.1710,  0.4468],
        [-0.3645, -0.0590],
        [-0.0883,  0.4495],
        [ 0.3032,  0.0831],
        [ 0.3365, -2.0816],
        [-1.3439,  0.1712],
        [-0.6996, -0.1786],
        [-0.7528, -0.6710],
        [ 0.3491, -0.0991],
        [ 0.6792,  1.0151],
        [-2.1685,  0.2037],
        [-0.4146, -1.0978],
        [ 1.2748,  1.8238]])

In [303]:
C[X].shape, C[X]

(torch.Size([228146, 3, 2]),
 tensor([[[ 0.9722,  0.1589],
          [ 0.9722,  0.1589],
          [ 0.9722,  0.1589]],
 
         [[ 0.9722,  0.1589],
          [ 0.9722,  0.1589],
          [-0.6390,  1.1987]],
 
         [[ 0.9722,  0.1589],
          [-0.6390,  1.1987],
          [-0.8901,  1.7070]],
 
         ...,
 
         [[ 1.2748,  1.8238],
          [ 1.2748,  1.8238],
          [-0.4146, -1.0978]],
 
         [[ 1.2748,  1.8238],
          [-0.4146, -1.0978],
          [ 1.2748,  1.8238]],
 
         [[-0.4146, -1.0978],
          [ 1.2748,  1.8238],
          [-2.1685,  0.2037]]]))

In [304]:
emb = C[X]
emb

tensor([[[ 0.9722,  0.1589],
         [ 0.9722,  0.1589],
         [ 0.9722,  0.1589]],

        [[ 0.9722,  0.1589],
         [ 0.9722,  0.1589],
         [-0.6390,  1.1987]],

        [[ 0.9722,  0.1589],
         [-0.6390,  1.1987],
         [-0.8901,  1.7070]],

        ...,

        [[ 1.2748,  1.8238],
         [ 1.2748,  1.8238],
         [-0.4146, -1.0978]],

        [[ 1.2748,  1.8238],
         [-0.4146, -1.0978],
         [ 1.2748,  1.8238]],

        [[-0.4146, -1.0978],
         [ 1.2748,  1.8238],
         [-2.1685,  0.2037]]])

In [305]:
W1 = torch.randn((6,100))
b1 = torch.randn(100)

In [306]:
W1.shape, b1.shape

(torch.Size([6, 100]), torch.Size([100]))

In [307]:
h = torch.tanh(emb.view(-1,6) @ W1 +b1)
h.shape

torch.Size([228146, 100])

In [308]:
W2 = torch.randn((100,27))
b2 = torch.randn(27)


In [309]:
logits = h @ W2 + b2
logits.shape

torch.Size([228146, 27])

In [310]:
counts = logits.exp()

In [311]:
prob = counts/counts.sum(1,keepdim=True)

In [312]:
prob.shape

torch.Size([228146, 27])

In [313]:
prob

tensor([[3.2049e-06, 7.2644e-02, 2.8463e-03,  ..., 2.5457e-05, 2.0399e-11,
         3.4916e-10],
        [1.8140e-08, 2.7558e-01, 1.7982e-01,  ..., 1.1461e-09, 3.6077e-06,
         1.3734e-09],
        [2.5777e-12, 2.6455e-06, 1.0737e-08,  ..., 3.9384e-14, 2.5888e-04,
         5.7899e-02],
        ...,
        [5.4611e-13, 2.5076e-06, 7.6193e-01,  ..., 1.3762e-04, 9.1527e-10,
         5.8979e-07],
        [3.4186e-15, 1.0697e-07, 6.7391e-13,  ..., 1.6643e-16, 2.9074e-17,
         6.7131e-14],
        [6.8966e-06, 3.3712e-03, 3.0270e-06,  ..., 4.9322e-07, 1.8508e-05,
         1.9343e-04]])

In [314]:
Y

tensor([ 5, 13, 13,  ..., 26, 24,  0])

In [315]:
#rewrite everything to make it reuseable

In [316]:
X.shape, Y.shape

(torch.Size([228146, 3]), torch.Size([228146]))

In [317]:
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27,2), generator=g)
W1 = torch.randn((6,100), generator=g)
b1 = torch.randn(100, generator=g)
W2 = torch.randn((100,27), generator=g)
b2 = torch.randn(27,generator=g)
parameters = [C, W1, b1, W2, b2]

In [318]:
sum(p.nelement() for p in parameters) #number of parameters

3481

In [319]:
for p in parameters:
  p.requires_grad = True

In [330]:
for _ in range(100):

  #minibatch construct
  ix = torch.randint(0,X.shape[0],(32,)) #batch size = 32
  #forward path
  emb = C[X[ix]] #(32, 3, 2)
  h = torch.tanh(emb.view(-1,6) @ W1 + b1) #(32,100)
  logits = h @ W2 + b2 #(32,27)
  #counts = logits.exp()
  #prob = counts / counts.sum(1, keepdim= True)
  #loss = -prob[torch.arange(32), Y].log().mean()
  loss = F.cross_entropy(logits, Y[ix]) #Using cross_entropy is preferred.
  print(loss.item()) #these are the loss in just 1 mini batch

  #backward path
  for p in parameters:
    p.grad = None
  loss.backward()
  #update
  for p in parameters:
    p.data += -0.1 * p.grad #learning rate = 0.1


2.9191455841064453
4.25274133682251
4.297205924987793
3.7640435695648193
4.127415180206299
4.137062072753906
3.5358126163482666
3.8467729091644287
2.981668710708618
3.3077471256256104
4.000509262084961
3.691378593444824
2.9315080642700195
4.427433967590332
3.899531364440918
3.598517656326294
3.683345317840576
2.957733392715454
4.336640357971191
3.300666093826294
3.3402507305145264
3.5568649768829346
3.716810703277588
3.9330687522888184
3.5113322734832764
3.449847936630249
3.247464656829834
3.9812285900115967
3.5941033363342285
3.747068166732788
4.042686939239502
3.745656728744507
3.1761038303375244
2.9467175006866455
3.172938108444214
3.22823429107666
3.2678611278533936
4.573258876800537
3.5203895568847656
3.0989387035369873
2.769001007080078
4.067061424255371
2.887908458709717
3.0579452514648438
3.632133960723877
3.495676279067993
4.3440842628479
3.6374285221099854
3.662851333618164
3.5675840377807617
3.1117844581604004
2.8441085815429688
3.3341073989868164
3.348586082458496
3.0525193

In [336]:
#evaluate the loss in all records
emb = C[X] #([228146, 3, 2])
h = torch.tanh(emb.view(-1,6) @ W1 + b1) #([228146, 100])
logits = h @ W2 + b2 #(228146, 27)
loss = F.cross_entropy(logits,Y)
loss

tensor(3.1296, grad_fn=<NllLossBackward0>)